### Importing Necessary Libraries

In [1]:
import math
import time
import cv2
import numpy as np
from pylab import array

### Network Initialization with body_25

In [3]:
"""
# COCO
protoFile = "models/pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "models/pose/coco/pose_iter_440000.caffemodel"
nPoints = 18
POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

# MPI
protoFile = "models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "models/pose/mpi/pose_iter_160000.caffemodel"
nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
"""

protoFile = "models/pose/body_25/pose_deploy.prototxt"
weightsFile = "models/pose/body_25/pose_iter_584000.caffemodel"
nPoints = 25
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
print(net)

< cv2.dnn.Net 00000188E3B132B0>


### Testing camera

In [4]:
# 1 is USB webcam
cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)

print(cap)

while(True):
    # Capture the video frame by frame
    ret, frame = cap.read()

    cv2.imshow('frame', frame)  # Display the resulting frame
    cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)  # Make it appear on top
   
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()

# Destroy all the windows
cv2.destroyAllWindows()

< cv2.VideoCapture 00000188E4345810>


### ⚠️⚠️⚠️ Reading in the frames ⚠️⚠️⚠️

In [5]:
# Give some time for setup
time.sleep(4)

FRAMES_READ = 60*5
FRAMES_SKIP = 5
frames = []

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)  # Apparently CAP_DSHOW makes it faster

for i in range(FRAMES_READ):
    # Not adding the first iteration of frames because camrea adjusts to lighting
    if i % FRAMES_SKIP == 0 and i != 0:
        frames.append(frame)

    ret, frame = cap.read()

    cv2.imshow("frame", frame)
    cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Get number of frames and size of `FRAMES` array in MB
print(len(frames), array(frames).nbytes/(1024*1024))

59 51.85546875


### Checking the Cached Video

In [6]:
LOOP = 1

for _ in range(LOOP):
    for i, frame in enumerate(frames, start=1):
        cv2.imshow("frame", frame)
        cv2.waitKey(0)  # Doesn't play automatically, hit any key to move to next frame
        cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)

        print(f"Frame {i}")

cv2.waitKey(0)
cv2.destroyAllWindows()

Frame 1
Frame 2
Frame 3
Frame 4
Frame 5
Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 16
Frame 17
Frame 18
Frame 19
Frame 20
Frame 21
Frame 22
Frame 23
Frame 24
Frame 25
Frame 26
Frame 27
Frame 28
Frame 29
Frame 30
Frame 31
Frame 32
Frame 33
Frame 34
Frame 35
Frame 36
Frame 37
Frame 38
Frame 39
Frame 40
Frame 41
Frame 42
Frame 43
Frame 44
Frame 45
Frame 46
Frame 47
Frame 48
Frame 49
Frame 50
Frame 51
Frame 52
Frame 53
Frame 54
Frame 55
Frame 56
Frame 57
Frame 58
Frame 59


### Pose Processing

In [10]:
# The actual body parts
body_parts_dict = {
    0: "Nose",
    1: "Neck",
    2: "RShoulder",
    3: "RElbow",
    4: "RWrist",
    5: "LShoulder",
    6: "LElbow",
    7: "LWrist",
    8: "MidHip",
    9: "RHip",
    10: "RKnee",
    11: "RAnkle",
    12: "LHip",
    13: "LKnee",
    14: "LAnkle",
    15: "REye",
    16: "LEye",
    17: "REar",
    18: "LEar",
    19: "LBigToe",
    20: "LSmallToe",
    21: "LHeel",
    22: "RBigToe",
    23: "RSmallToe",
    24: "RHeel",
    25: "Background"
}

# Some are excluded
head_and_neak = {0, 1}  # 0.07
trunk = {8, 9, 12}  # 0.43
upper_limbs = {2, 3, 4, 5, 6, 7}  # 0.13
lower_limbs = {10, 11, 13, 14}  # 0.37
union = head_and_neak | trunk | upper_limbs | lower_limbs

loop = []
center_of_grav_loc = []

for j, frame in enumerate(frames, start=1):
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    threshold = 0.1

    inWidth = 640//3
    inHeight = 480//3
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

    # Bottleneck
    net.setInput(inpBlob)
    output = net.forward()
    
    H = output.shape[2]
    W = output.shape[3]

    # Empty list to store the detected keypoints
    points = []

    x_sum = 0
    y_sum = 0

    for i in range(nPoints):
        # If the point is excluded from the minimum points required
        if i not in union:
            continue

        # Weighting of an individual body part to be assigned
        weight = None

        # Identify the set its in
        if i in head_and_neak:
            weight = 0.07/len(head_and_neak)
        elif i in trunk:
            weight = 0.43/len(trunk)
        elif i in upper_limbs:
            weight = 0.13/len(upper_limbs)
        elif i in lower_limbs:
            weight = 0.37/len(lower_limbs)

        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        # Weighted average
        x_sum += x*weight
        y_sum += y*weight

        if prob > threshold or 1:
            # Body part location and text
            cv2.circle(frameCopy, (int(x), int(y)), 4, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, f"{i}: {body_parts_dict[i]}", (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))

        else :
            points.append(None)
    
    # Draw center of gravity as a green circle and add it to array for tracking
    cv2.circle(frameCopy, (int(x_sum), int(y_sum)), 10, (0, 255, 0), thickness=-1, lineType=cv2.FILLED)
    center_of_grav_loc.append((int(x_sum), int(y_sum)))

    """
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    """

    # Adding annotated frame for COG analysis
    loop.append(frameCopy)

    print(f"Processed frame {j}")

print("Done")

Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed frame 51
Processed frame 52
Processed frame 53
Pr

### Looping the Tracked COG

In [11]:
LOOP = 10

for _ in range(LOOP):
    for i, frame in enumerate(loop):
        cv2.imshow("frame", frame)
        cv2.waitKey(0)
        cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)
  
        print(f"Frame {i}")

cv2.waitKey(0)
cv2.destroyAllWindows()

Frame 0
Frame 1
Frame 2
Frame 3
Frame 4
Frame 5
Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 16
Frame 17
Frame 18
Frame 19
Frame 20
Frame 21
Frame 22
Frame 23
Frame 24
Frame 25
Frame 26
Frame 27
Frame 28
Frame 29
Frame 30
Frame 31
Frame 32
Frame 33
Frame 34
Frame 35
Frame 36
Frame 37
Frame 38
Frame 39
Frame 40
Frame 41
Frame 42
Frame 43
Frame 44
Frame 45
Frame 46
Frame 47
Frame 48
Frame 49
Frame 50
Frame 51
Frame 52
Frame 53
Frame 54
Frame 55
Frame 56
Frame 57
Frame 58


KeyboardInterrupt: 

### COG Distance

In [ ]:
tot = 0

for i, j in zip(center_of_grav_loc[:-1], center_of_grav_loc[1:]):
    dist = math.dist(i, j)
    tot += dist
    print(f"{i} -> {j}: {dist} pixels")

print(tot, "pixels")

### Random Testing

In [9]:
frames[0].shape

(480, 640, 3)